# Lab 2 - Introduction to Pandas

In [ ]:
# dependencies
import sys
from datetime import datetime, date
import json
from IPython.core.display import HTML, display
import matplotlib.pyplot as plt
%matplotlib inline

## Introduction
[Pandas](http://pandas.pydata.org/), [NumPy](http://www.numpy.org/) and [SciPy](https://www.scipy.org/) are popular libraries in Python for data analysis.

This notebook will give you a very quick overview of some of its capabilities.

For more details:
* [A 10min quickstart](http://pandas.pydata.org/pandas-docs/stable/10min.html)
* [Pandas documentation](http://pandas.pydata.org/pandas-docs/version/0.18.0/)

As for the `requests` module, `pandas` has to be imported in the Python namespace

In [ ]:
import pandas

Let's create a dummy data set:

In [ ]:
tracks = [
    {'name': 'track1', 'popularity': 10, 'genre': 'rock', 'release_date': '2014-01-01', 'custom': ''},
    {'name': 'track2', 'popularity': 84, 'genre': 'punk', 'release_date': '2014-01-01', 'custom': ''},
    {'name': 'track3', 'popularity': 65, 'genre': 'reggae', 'release_date': '2014-02-01', 'custom': ''},
    {'name': 'track4', 'popularity': 62, 'genre': 'punk', 'release_date': '2014-01-01', 'custom': ''},
    {'name': 'track5', 'popularity': 5, 'genre': 'rock', 'release_date': '2014-01-01', 'custom': ''},
    {'name': 'track6', 'popularity': 13, 'genre': 'reggae', 'release_date': '2014-01-01', 'custom': ''},
    {'name': 'track7', 'popularity': 56, 'genre': 'rock', 'release_date': '2012-01-01', 'custom': ''},
    {'name': 'track8', 'popularity': 53, 'genre': 'punk', 'release_date': '2014-12-01', 'custom': ''},
    {'name': 'track9', 'popularity': 76, 'genre': 'rock', 'release_date': '2014-01-01', 'custom': ''},
    {'name': 'track10', 'popularity': 72, 'genre': 'rock', 'release_date': '2014-01-01', 'custom': ''},
    {'name': 'track11', 'popularity': 75, 'genre': 'rock', 'release_date': '2014-01-01', 'custom': ''},
    {'name': 'track12', 'popularity': 76, 'genre': 'rock', 'release_date': '1967-01-01', 'custom': ''},
    {'name': 'track13', 'popularity': 2, 'genre': 'rock', 'release_date': '2014-01-01', 'custom': ''},
    {'name': 'track14', 'popularity': 15, 'genre': 'rock', 'release_date': '2014-01-01', 'custom': ''},    
]

### Creating a DataFrame
A [*DataFrame*](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) is one of the main data structure in Pandas. 

It is a 2 dimensional data structure with columns of potentially different types with indexes axes (rows and columns). 

Among the various possibilities, a Pandas's DataFrame can be created easily from a list of objects using `json_normalize`

In [ ]:
from pandas.io.json import json_normalize
df = json_normalize(tracks)

The `head()` method can be used on a DataFrame object to return the `n` first rows (default is 5)

In [ ]:
df.head(2) 

The types associated with the columns are:

In [ ]:
df.dtypes

As you can see, this DataFrame is made of 5 columns with different types (string & integer, the date is in this example encoded as a string but we could have use the `datetime` or `date` type from the `datetime` module).

From this DataFrame we can perform some operations
 * get the number of rows/records:

In [ ]:
# len can be used on a data frame to get the number of rows
print('Num records:', len(df))

# the shape properties contains (number of rows, number of columns)
print('DataFrame shape:', df.shape)

* select some specific columns

In [ ]:
# either using a pyhon property
df.name

In [ ]:
# or by its name
df['genre']

* select some specific rows

In [ ]:
df[3:5]

On numerical fields, Pandas can compute some basic statistics on the DataFrame with the `describe` function

In [ ]:
df.describe()

For more details on the DataFrame object you can use the `help` function and also consult http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html

In [ ]:
help(df)

### Grouping
Let's count the number of tracks grouped by genre.

In [ ]:
df.groupby('genre')

The `DataFrameGroupBy` is creating the grouping and can be used in several ways:
* iterating through the groups and there respective DataFrame:

In [ ]:
for key, group in df.groupby('genre'):
    print('Genre: %s' % key)
    print('DataFrame:')
    display(group)

* get the groups and the index of the associated records:

In [ ]:
print(df.groupby('genre').groups)

* get the size of each groups:

In [ ]:
df.groupby('genre').size()

* apply some aggregation functions for each group:

In [ ]:
# count of records
display(df.groupby('genre').count())

# avg for numeric fields
display(df.groupby('genre').mean())

* We can also display this now using the `.plot()` function:

In [ ]:
df.groupby('genre').size().plot(kind='bar', title='Repartition of tracks by genre', figsize=(5,5))

Grouping may be performed on several columns and an aggregation function being used.

The following counts the number of tracks grouped by release date and genre

In [ ]:
df.groupby(['release_date', 'genre']).count()

and this ones compute the average popularity:

In [ ]:
df.groupby(['release_date', 'genre']).mean()

Sometime you may want to pivot the ouptut to get a columnar representation of some groups

In [ ]:
df.groupby(['release_date', 'genre']).mean().unstack('genre')

and we can replace undefined value with a default value:

In [ ]:
df.groupby(['release_date', 'genre']).mean().unstack('genre').fillna(0)

which we can use to plot the popularity by year and type:

In [ ]:
ax = df.groupby(['release_date', 'genre']).mean().unstack('genre').fillna(0).plot(kind='bar', stacked=True)
ax.set_xlabel('Release date')
ax.set_ylabel('Popularity')

### Filtering
Filters can easily be applied on a DataFrame.

For instance, to filter all records with a popularity greater than 60 and then group them by genre:

In [ ]:
ax = df[df.popularity > 60]\
    .groupby('genre').size()\
    .plot(kind='pie', \
          title='Repartition of genre for tracks with popularity > 60', \
          figsize=(5,5))

We could also plot the distribution of popularity for rock tracks

In [ ]:
ax = df[df.genre == 'rock']['popularity'].hist(bins=10, figsize=(10, 5))
ax.set_xlabel('Popularity')
ax.set_ylabel('Count of tracks')

### Deriving new data

Functions can be applied to columns to generate new values or columns.

For instance, we may want to group the tracks by year of release and not the day of release. This could be done for instance by:
* creating a new `year` feature
* applying the group on the feature

In [ ]:
df['release_date'].apply(lambda x: x.split('-')[0])

We can extend our DataFrame with this new column:

In [ ]:
df['year'] = df['release_date'].apply(lambda x: x.split('-')[0])

In [ ]:
df.head(2)

And now we can display the repartition of genre by year of release

In [ ]:
ax = df.groupby(['year', 'genre'])['name'].count()\
    .unstack('genre').fillna(0)\
    .plot(kind='bar', stacked=True)
ax.set_xlabel('Year')
ax.set_ylabel('Number of tracks')

You can also drop columns for the DataFrame to clean up the data

In [ ]:
df.drop('custom', axis=1, inplace=True)
df.head(2)

# Exercice - User profiles analysis

Using the [LastFm 360K dataset](http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/lastfm-360K.html), you'll be doing some analysis of a subset of LastFM anonymyzed users profiles.

This data sets containing for each user:
* id (SHA1)
* gender
* age
* country
* signup date


    user-mboxsha1 \t gender (m|f|empty) \t age (int|empty) \t country (str|empty) \t signup (date|empty)


You can download the file at the following address:

    https://s3-eu-west-1.amazonaws.com/scimus-data/lab2/usersha1-profile.tsv.gz
    
Put it somewhere in your Home directory, eg:
    
    mkdir scimus-data
    curl -O https://s3-eu-west-1.amazonaws.com/scimus-data/lab2/usersha1-profile.tsv.gz
    gunzip usersha1-profile.tsv.gz

By analysing the data set with Pandas, answer the following questions:

* Load the data set into a DataFrame. Some hints:
 * use the `read_csv` function from the `pandas` package
 * the fields are separated by a tabular character (`\t`)
 * there is no header in the file
* Create a new column by transforming the `signup` string into a proper python `date` object. Some hints:
 * use the `strptime` method of `datetime` (see the Python documentation for the string [format](https://docs.python.org/3.5/library/datetime.html#strftime-strptime-behavior))
 * `datetime` python objects can be transformed to `date` with the `.date()` method
* Display the total number of signups by month
* Study the repartition of signups (total and monthly) by gender, country and age. Some hints:
 * consider bucketing age by bucket of 10 years
 * display the monthly signups for the 10 top countries by count of signups
* Study the age data, what can you observe in the source data?

In [ ]:
# Your code / answers here